In [20]:
import os

import gensim
import lucem_illud
import pandas as pd
from tqdm import tqdm

%matplotlib inline
tqdm.pandas()

### Constants and Utility Functions

In [21]:
# Constants
MAX_CHAR_LEN = 5000000

# <font color="red">*Pitch Your Project*</font>
<font color="red">In the three cells immediately following, describe **WHAT** you are 
planning to analyze for your final project (i.e., texts, contexts and the social game, 
world and actors you intend to learn about through your analysis) (<200 words), **WHY** 
you are going to do it (i.e., why would theory and/or the average person benefit from 
knowing the results of your investigation) (<200 words), and **HOW** you plan to investigate 
it (i.e., what are the approaches and operations you plan to perform, in sequence, to yield 
this insight) (<400 words).

## ***What? (<200 words)***
For our project, Chanteria Milner and I will analyze congressional and Supreme Court 
abortion-related legislation. Particularly for the congressional legislation, we will explore 
all legislation from 1973 until 2024. We sourced this legislation from the congress.gov 
legislation search, where we filtered for any legislation within this period that could 
have become bills and included the following keywords in the bill text or summary: 
'abortion,' 'reproduction,' or 'reproductive health care.' For the SCOTUS abortion 
legislation, we targeted SCOTUS decisions outlined on supreme.justia.com, which provides a 
list of abortion-relevant SCOTUS decisions from 1965-2022. Through this analysis, we 
plan to uncover the ways that the legislative abortion discourse has changed over time and 
the prominent congressional bills that have occurred throughout subsequent pieces of legislation. 
We will supplement this analysis with legislation in its political history by taking 
note of the political affiliations of congress members, SCOTUS justices, and the presidency.

Chanteria Milner: [Link](https://github.com/chanteriam)
Michael Plunkett: [Link](https://github.com/michplunkett)\
Congressional source: [Link](https://www.congress.gov/advanced-search/legislation?congressGroup%5B%5D=0&congresses%5B%5D=118&congresses%5B%5D=117&congresses%5B%5D=116&congresses%5B%5D=115&congresses%5B%5D=114&congresses%5B%5D=113&congresses%5B%5D=112&congresses%5B%5D=111&congresses%5B%5D=110&congresses%5B%5D=109&congresses%5B%5D=108&congresses%5B%5D=107&congresses%5B%5D=106&congresses%5B%5D=105&congresses%5B%5D=104&congresses%5B%5D=103&congresses%5B%5D=102&congresses%5B%5D=101&congresses%5B%5D=100&congresses%5B%5D=99&congresses%5B%5D=98&congresses%5B%5D=97&congresses%5B%5D=96&congresses%5B%5D=95&congresses%5B%5D=94&congresses%5B%5D=93&legislationNumbers=&restrictionType=field&restrictionFields%5B%5D=allBillTitles&restrictionFields%5B%5D=summary&summaryField=billSummary&enterTerms=%22reproductive+health+care%22%2C+%22reproduction%22%2C+%22abortion%22&legislationTypes%5B%5D=hr&legislationTypes%5B%5D=hjres&legislationTypes%5B%5D=s&legislationTypes%5B%5D=sjres&public=true&private=true&chamber=all&actionTerms=&legislativeActionWordVariants=true&dateOfActionOperator=equal&dateOfActionStartDate=&dateOfActionEndDate=&dateOfActionIsOptions=yesterday&dateOfActionToggle=multi&legislativeAction=Any&sponsorState=One&member=&sponsorTypes%5B%5D=sponsor&sponsorTypeBool=OR&dateOfSponsorshipOperator=equal&dateOfSponsorshipStartDate=&dateOfSponsorshipEndDate=&dateOfSponsorshipIsOptions=yesterday&committeeActivity%5B%5D=0&committeeActivity%5B%5D=3&committeeActivity%5B%5D=11&committeeActivity%5B%5D=12&committeeActivity%5B%5D=4&committeeActivity%5B%5D=2&committeeActivity%5B%5D=5&committeeActivity%5B%5D=9&satellite=null&search=&submitted=Submitted)
SCOTUS source: [Link](https://supreme.justia.com/cases-by-topic/abortion-reproductive-rights/)

## ***Why? (<200 words)***
This analysis, in general, will give us an understanding of how abortion discourse has
changed over time, and specifically, what arguments were used in the passing of the 
1973 Roe v. Wade decision--and with it, the assertion of the constitutional right to 
abortion and its eventual reversal in the 2022 Dobbs v. Jackson decision. 
From this, we will be able to uncover what political mechanisms were at play that 
enabled this regression in legislation, and how such significant cases went on to influence 
congressional legislation that followed. The average person will, in general, be able 
to understand how sensitive practices are discussed in the political sphere and what 
aspects in particular are targeted for protection or not. Moreover, this analysis 
will allow individuals who want to write reproductive healthcare legislation to 
understand what arguments are more likely to work over others within certain 
political contexts.

## ***How? (<400 words)***


## <font color="red">*Pitch Your Sample*</font>
<font color="red">In the cell immediately following, describe the rationale behind 
your proposed sample design for your final project. What is the social game, social 
work, or social actors you about whom you are seeking to make inferences? What are 
its virtues with respect to your research questions? What are its limitations? What 
are alternatives? What would be a reasonable path to "scale up" your sample for 
further analysis (i.e., high-profile publication) beyond this class?.

## ***Which words? (<300 words)***
Starting in the early 1970s with the adoption of the Hyde Amendment, the amendment 
that prevented federal funding of abortions through Medicaid, anti-abortion advocates 
have attempted to reverse the Roe decision through a multitude of avenues. These efforts 
eventually culminated in the Dobbs v. Jackson decision in 2022. The elevation of the Dobbs 
v. Jackson case was not something that happened by random chance but instead was the result 
of decades of concerted efforts by anti-abortion advocates, think tanks, and political 
action committees. In our project, we intend to see how the social game of these groups 
probing for judicial vulnerabilities changed over time and the language that made up 
those attempts. The actors in this study are primarily the congressional legislators 
and supreme court justices, whom we will study via their decisions and proposed pieces 
of legislation, respectively. This analysis will give us a direct look into the 
terminology used to overturn Roe in its successful and unsuccessful cases.

Our analysis in this project will undoubtedly give us some valuable insights, but it 
will not allow us to create the entire picture. The sources we are using only capture 
the result of the efforts made by anti-abortion advocates, etc. A more complete design 
of this analysis would take documents from conservative think tanks like the Heritage 
Foundation, Family Research Council, Moral Majority, etc. Taking in documents from 
these organizations and other similarly minded ones would give us insight into the 
documents upstream from the legislative and decision documents we're currently 
assessing. To scale this project up for a high-profile journal, we could get 
documents from those organizations listed and look for similarities between them 
and the documents we're currently examining to see the complete evolution of the 
language from its initial planning to attempts at federal implementation.

## <font color="red">*Exercise 1*</font>

<font color="red">Construct cells immediately below this that embed documents 
related to your final project using at least two different specification of 
`word2vec` and/or `fasttext`, and visualize them each with two separate 
visualization layout specifications (e.g., TSNE, PCA). Then interrogate 
critical word vectors within your corpus in terms of the most similar words, 
analogies, and other additions and subtractions that reveal the structure of 
similarity and difference within your semantic space. What does this pattern 
reveal about the semantic organization of words in your corpora? Which 
estimation and visualization specification generate the most insight and 
appear the most robustly supported and why?

<font color="red">***Stretch***: Explore different vector calculations 
beyond addition and subtraction, such as multiplication, division or some 
other function. What does this exploration reveal about the semantic 
structure of your corpus?

In [38]:
# Load the congressional legislation
congressional_leg_df = pd.read_feather("data/tokenized_congress_legislation.feather")
congressional_leg_df.head()

,legislation number,url,congress,title,date proposed,amends amendment,latest summary,congress_num,bill_type,bill_num,api_url,text_url,raw_text,cleaned_text,cleaned_summary,kmeans_predictions,fcluster_predictions,tokenized_text,normalized_tokens,reduced_tokens
0,H.R. 2907,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Let Doctors Provide Reproductive Health Care Act,NaN,NaN,<p><strong>Let Doctors Provide Reproductive H...,118,hr,2907,https://api.congress.gov/v3/bill/118/hr/2907/t...,https://www.congress.gov/118/bills/hr2907/BILL...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Let Doctors Provide Reproductive Health Care A...,29,2,"[time, enforcement, or, protective, considerat...","[publishing, time, enforcement, protective, co...","[enforcement, consideration, specific, affecte..."
1,S. 1297,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),Let Doctors Provide Reproductive Health Care Act,NaN,NaN,<p><strong>Let Doctors Provide Reproductive H...,118,s,1297,https://api.congress.gov/v3/bill/118/s/1297/te...,https://www.congress.gov/118/bills/s1297/BILLS...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Let Doctors Provide Reproductive Health Care A...,29,2,"[Murphy, time, Hirono, enforcement, Welch, or,...","[publishing, time, van, enforcement, mrs, prot...","[enforcement, specific, affected, defined, sin..."
2,H.R. 4901,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Reproductive Health Care Accessibility Act,NaN,NaN,<p><strong>Reproductive Health Care Accessibi...,118,hr,4901,https://api.congress.gov/v3/bill/118/hr/4901/t...,https://www.congress.gov/118/bills/hr4901/BILL...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Reproductive Health Care Accessibility Act Thi...,7,2,"[experience, g, Community, throughout, partici...","[publishing, experience, g, participating, tim...","[participating, tribes, centers, consideration..."
3,S. 2544,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),Reproductive Health Care Accessibility Act,NaN,NaN,<p><strong>Reproductive Health Care Accessibi...,118,s,2544,https://api.congress.gov/v3/bill/118/s/2544/te...,https://www.congress.gov/118/bills/s2544/BILLS...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Reproductive Health Care Accessibility Act Thi...,7,2,"[experience, 2544, g, Community, throughout, p...","[publishing, experience, g, participating, tim...","[participating, tribes, centers, consideration..."
4,H.R. 4147,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Reproductive Health Care Training Act of 2023,NaN,NaN,None,118,hr,4147,https://api.congress.gov/v3/bill/118/hr/4147/t...,https://www.congress.gov/118/bills/hr4147/BILL...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,None,0,2,"[g, identification, minority, time, centers, o...","[improve, act, public, pensions, cited, publis...","[improve, establish, number, identification, p..."


In [40]:
# Since the tokenization process takes so much computational power/time, I put it
# in a feather file to avoid the time when re-running this analysis.
legislation_feather_path = "data/fully_tokenized_congress_legislation.feather"
if os.path.isfile(legislation_feather_path):
    congressional_leg_df = pd.read_feather(legislation_feather_path)
else:
    # WARNING: This step takes about 120 minutes, so don't run it unless you need to.
    congressional_leg_df["tokenized_sentences"] = congressional_leg_df[
        "cleaned_text"
    ].progress_apply(
        lambda x: [
            lucem_illud.word_tokenize(s, MAX_LEN=MAX_CHAR_LEN)
            for s in lucem_illud.sent_tokenize(x)
        ]
    )
    congressional_leg_df["normalized_sentences"] = congressional_leg_df[
        "tokenized_sentences"
    ].apply(
        lambda x: [
            lucem_illud.normalizeTokens(s, lemma=False, MAX_LEN=MAX_CHAR_LEN) for s in x
        ]
    )
    congressional_leg_df.to_feather(legislation_feather_path)

congressional_leg_df.shape

  0%|          | 0/1243 [00:00<?, ?it/s]/Users/michaelp/Documents/GitHub/computational-content-analysis/.venv/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
100%|██████████| 1243/1243 [1:24:49<00:00,  4.09s/it]  


(1243, 22)

In [41]:
# Load saved Word2Vec if it exists
word2vec_file_path = "data/congressional_leg_w2v"
if os.path.isfile(word2vec_file_path):
    congressional_leg_w2v = gensim.models.word2vec.Word2Vec.load(word2vec_file_path)
else:
    congressional_leg_w2v = gensim.models.word2vec.Word2Vec(
        congressional_leg_df["normalized_sentences"].sum(), sg=0
    )
    congressional_leg_w2v.save(word2vec_file_path)

In [42]:
# Going to look at words commonly associated with abortion
congressional_leg_w2v.wv.most_similar(["abortion", "healthcare", "health"])

[('gynecological', 0.5437756180763245),
 ('patients', 0.5378872752189636),
 ('obstetrician', 0.5100891590118408),
 ('obstetrical', 0.5008599162101746),
 ('patient', 0.49554017186164856),
 ('care', 0.48337480425834656),
 ('pediatric', 0.47294947504997253),
 ('osteoporosis', 0.47060659527778625),
 ('prenatal', 0.467769980430603),
 ('postpartum', 0.45360267162323)]

## <font color="red">*Exercise 2*</font>

<font color="red">Construct cells immediately below this that embed 
documents related to your final project using `doc2vec`, and explore 
the relationship between different documents and the word vectors you 
analyzed in the last exercise. Consider the most similar words to 
critical documents, analogies (doc _x_ + word _y_), and other additions 
and subtractions that reveal the structure of similarity and difference 
within your semantic space. What does this pattern reveal about the 
documentary organization of your semantic space?

## <font color="red">*Exercise 3*</font>

<font color="red">Construct cells immediately below this that embed 
documents related to your final project, then generate meaningful 
semantic dimensions based on your theoretical understanding of the 
semantic space (i.e., by subtracting semantically opposite word vectors) 
and project another set of word vectors onto those dimensions. Interpret 
the meaning of these projections for your analysis. Which of the dimensions 
you analyze explain the most variation in the projection of your words and why?

<font color="red">***Stretch***: Average together multiple antonym pairs to 
create robust semantic dimensions. How do word projections on these robust 
dimensions differ from single-pair dimensions?

## <font color="red">*Exercise 4*</font>

<font color="red">Construct cells immediately below this that align 
word embeddings over time or across domains/corpora. Interrogate the 
spaces that result and ask which words changed most and least over the 
entire period or between contexts/corpora. What does this reveal about 
the social game underlying your space?